# NoteBook de Gautier Hoarau

## Sommaire
1. [Importations et Chargement des Donnees](#1-importations-et-chargement-des-donnees)
2. [Pretraitement des Donnees](#2-pretraitement-des-donnees)
3. [Tokenization et Vectorisation](#3-tokenization-et-vectorisation)
4. [Construction du Modele](#4-construction-du-modele)
5. [Entrainement et Evaluation](#5-entrainement-et-evaluation)
6. [Pipeline de Classification](#6-pipeline-de-classification)
7. [Exemple test](#7-exemple-test)

## 1. Importations et Chargement des Donnees

In [ ]:
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import drive

In [ ]:
# Télécharger les stopwords
nltk.download('punkt')     # Tokenisation des mots
nltk.download('stopwords') # Liste des mots vides

In [ ]:
# Chargement des données
drive.mount('/content/drive', force_remount=True)
data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')
data.head()

# Liste des colonnes représentant les différentes catégories de toxicité
toxic_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Création d'une nouvelle colonne 'label' dans le DataFrame, qui prend la valeur maximale parmi les colonnes de toxicité pour chaque ligne
# Cela permet de marquer un commentaire comme toxique si l'une des catégories de toxicité a la valeur la plus élevée: 1 si le commentaire est toxique, 0 sinon
data['label'] = data[toxic_columns].max(axis=1)

# Affichage des premières lignes du DataFrame avec les colonnes 'comment_text' et 'label' pour vérifier que les résultats sont correctement associés aux commentaires
print(data[['comment_text', 'label']].head())

## 2. Pretraitement des Donnees

In [ ]:
# Définition de la fonction de nettoyage du texte
def clean_text(text):
    # Conversion du texte en minuscules et suppression des ponctuations
    text = text.lower()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    return text

# Fonction pour supprimer les mots vides d'un texte
def remove_stopwords(text):
    tokens = word_tokenize(text)    # Découpe du texte en mots
    stop_words = set(stopwords.words('english')) # Liste des mots vides
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Filtrage des mots vides
    return ' '.join(filtered_tokens) # Réassemble les mots restants

# Appliquer le prétraitement
data['cleaned_text'] = data['comment_text'].apply(clean_text).apply(remove_stopwords)
data.head()

## 3. Tokenization et Vectorisation

In [ ]:
# Définition du nombre maximum de mots et de la longueur maximale des séquences
max_words = 10000
max_len = 100

# Initialisation du Tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
# Apprentissage du Tokenizer sur le texte nettoyé
tokenizer.fit_on_texts(data['cleaned_text'])
# Conversion des textes
sequences = tokenizer.texts_to_sequences(data['cleaned_text'])
# Application du padding pour uniformiser la longueur des séquences
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

## 4. Construction du Modele

In [ ]:
# Définition du modèle simple de réseau de neurones avec Keras
model = Sequential([
    # Couche d'embedding pour transformer les indices des mots en vecteurs
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    # Couche LSTM avec 64 neurones
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    # Couche LSTM avec 32 neurones
    LSTM(32),
    # Couche dense avec 16 neurones et activation ReLU
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## 5. Entrainement et Evaluation

In [ ]:
# Entraînement du modèle avec les données
X = padded_sequences
y = np.array(data['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Boucle pour évaluer le modèle pour chaque type de toxicité
for toxic_type in toxic_columns:
    print(f"Évaluation pour la catégorie : {toxic_type}")
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    # Prédiction des résultats sur l'ensemble de test (binaire : 0 ou 1)
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    # Affichage du rapport de classification pour chaque catégorie
    print(classification_report(y_test, y_pred))
    # Affichage de la matrice de confusion pour évaluer les prédictions
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    # Séparation des évaluations de chaque catégorie
    print("\n" + "-"*50 + "\n")

In [ ]:
 # Affichage des courbes de perte et de précision pour l'entraînement et la validation
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

## 6. Pipeline de Classification

In [ ]:
# Fonction de prétraitement du texte et de prédiction de la toxicité
def preprocess_and_predict(text, tokenizer, model, max_len=100):
    # Nettoyage du texte
    text_cleaned = clean_text(text)
    # Stopwords
    text_cleaned = remove_stopwords(text_cleaned)
    # Conversion du texte en séquence d'indices de mots
    sequence = tokenizer.texts_to_sequences([text_cleaned])
    # Vérifie si le texte est vide
    if len(sequence[0]) == 0:
        return "Inconnu (aucun mot reconnu)"
    # Padding
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')
    # Prédiction
    prediction = model.predict(padded_sequence)
    # Classification
    prob = prediction[0][0]
    classification = "Toxique" if prob > 0.5 else "Non toxique"
    # Retourne le résultat : "Toxique" si la probabilité est supérieure à 0.5, sinon "Non toxique"
    return f"{classification} (Score : {prob:.2f})"

## 7. Exemple test

In [ ]:
# Exemple de prédiction sur un commentaire
exemple = "Tu es stupide et méchant !"
# Prédiction du modèle final
resultat = preprocess_and_predict(exemple, tokenizer, model)  
print(f"Le commentaire est : {resultat}")

In [ ]:
exemple = "Bonjour, comment vas-tu ?"
resultat = preprocess_and_predict(exemple, tokenizer, model)
print(f"Le commentaire est : {resultat}")